## k-겹 교차검증을 사용한 모델 성능 평가
* 이 장의 주요 목적은 머신 러닝 모델을 구축하는 핵심 단계 중 하나인 처음 본 데이터에 대한 모델 성능을 추정하는 것.
* 홀드아웃 교차 검증(holdout cross-validation)
* k-겹 교차 검증(k-fold cross-validation)

### 홀드아웃 방법
* 모델 선택 : 주어진 분류 문제에서 튜닝할 파라미터의 최적 값을 선택해야 하는 것
* 모델 선택에 같은 테스트 세트를 반복해서 재사용하면 훈련 세트의 일부가 되는 셈이고 결국 모델은 과대적합된다. 많은 사람이 모델 선택을 위해 테스트 세트를 사용하지만 이는 좋은 머신러닝 작업방식이 아니다.
* 모델 선택에 홀드아웃 방법을 사용하는 가장 좋은 방법은 훈련 세트, 검증 세트, 테스트 세트 세개의 부분으로 나누는 것
![https://t1.daumcdn.net/cfile/tistory/994042405E24E8081C](https://t1.daumcdn.net/cfile/tistory/994042405E24E8081C)
* 홀드아웃 방법은 훈련 데이터를 훈련 세트와 검증 세트로 나누는 방법에 따라  성능 추정이 민감할 수 있다는 것이 단점이다.(ex.데이터셋이 작은 경우) 좀 더 안정적인 성능 추정 기법이 k-fold-validation

### k-겹 교차 검증
k-겹 교차검증에서는 중복을 허락하지 않고 훈련 데이터셋을 k개의 폴드로 랜덤하게 나눈다. k-1개의 폴드로 모델을 훈련하고 나머지 하나의 폴드로 성능을 평가한다. 이 과정을 k번 반복하여 k개의 모델과 성능 추정을 얻는다.
그 다음 서로 다른 독립적인 폴드에서 얻은 성능 추정을 기반으로 모델의 평균 성능을 계산한다.
만족스러운 하이퍼파라미터 값을 찾은 후에는 전체 훈련세트를 사용하여 모델을 다시 훈련한다. 그 다음 독립적인 테스트 세트를 사용하여 최종 성능 추정을 한다.
* k-겹 교차검증은 중복을 허락하지 않는 리샘플링 기법이기 때문에 모든 샘플 포인트가 훈련하는 동안 검증에 딱 한 번 사용되는 장점이 있다. 이로 인해 홀드아웃 방법보다 모델 성능의 추정에 분산이 낮다.
* 경험적으로 k=10이 가장 좋은 기본값이다. (가장 뛰어난 편향-분산 트레이드오프)
* 데이터가 적을 수록 k는 크게 하는 것이 좋다. 반대로 너무 큰 데이터는 k값이 작아도 평균 성능을 정확하게 추정할 수 있다.


* 계층적 k-겹 교차 검증(statified k-fold cross-validation) : 기존의 방법보다 좀 더 향상된 방법으로 더 나은 편향과 분산 추정을 만든다. 특히 클래스 비율이 동등하지 않을 때 계층적 교차 검증은 각 폴드에서 클래스 비율이 전체 훈련 세트에 있는 클래스 비율을 대표하도록 유지한다.

In [7]:
# 유방암 데이터셋

import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases'
                '/breast-cancer-wisconsin/wdbc.data', header = None)

In [8]:
# 인코딩
from sklearn.preprocessing import LabelEncoder
import numpy as np

X = df.loc[:,2:].values
y = df.loc[:,1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [9]:
# 훈련 테스트 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                   stratify = y, random_state = 1)
X.shape

(569, 30)

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [11]:
pipe_lr = make_pipeline(StandardScaler(),
                       PCA(n_components=2),
                       LogisticRegression(solver='liblinear',random_state = 1))

In [12]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [24]:
kfold = StratifiedKFold(n_splits=10, random_state=1).split(X_train,y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    print(train,test)
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('폴드 : %2d, 클래스 분포 : %s, 정확도 : %.3f' % (k+1, np.bincount(y_train[train]), score))

[ 45  46  47  48  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207
 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225
 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243
 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279
 280 281 282 283 284 285 286 287 288 289 290 291 29

C:\Users\sukwoo\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [20]:
print(np.mean(scores),np.std(scores))

0.9495169082125603 0.013854294239660376


In [23]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator = pipe_lr,
                        X=X_train,
                        y=y_train,
                        cv = 10,
                        n_jobs = 1) # njobs는 성능 평가에 사용될 cpu코어 수, -1이면 전체
print(scores)
print(np.mean(scores), np.std(scores))

[0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
0.9495169082125603 0.013854294239660376
